In [0]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('drive/My Drive/LSTM Fake News/train.csv')

df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20800 entries, 0 to 20799
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id      20800 non-null  int64 
 1   title   20242 non-null  object
 2   author  18843 non-null  object
 3   text    20761 non-null  object
 4   label   20800 non-null  int64 
dtypes: int64(2), object(3)
memory usage: 812.6+ KB


In [0]:

df= df.dropna()

X = df.drop('label',axis=1)
y = df.label

In [5]:
X.shape

(18285, 4)

In [6]:
import tensorflow as tf

tf.__version__

'2.2.0'

In [0]:

from tensorflow.keras.layers import Dense, Embedding, LSTM
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot

In [0]:
vocab_size = 5000

In [0]:
messages = X.copy()

In [0]:
messages.reset_index(inplace=True)

In [0]:
import nltk
import re
from nltk.corpus import stopwords

In [15]:
# Data Preprocessing
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [0]:
corpus = []
for i in range(len(messages)):
  #print(i)
  review = re.sub('[^a-zA-Z]', ' ', messages['title'][i])
  review = review.lower()
  review = review.split()
  review = [ps.stem(word) for word in review if word not in stopwords.words('english')]
  review = ' '.join(review)
  corpus.append(review)

In [19]:
len(corpus)

18285

In [0]:
onehot_rep = [one_hot(words, vocab_size) for words in corpus]

In [21]:
onehot_rep[:5]

[[242, 4462, 4210, 2930, 4913, 2641, 3230, 134, 1770, 3650],
 [4007, 3917, 490, 2881, 2113, 2127, 3479],
 [4905, 1399, 4612, 4964],
 [1559, 2312, 1795, 4178, 2038, 2410],
 [171, 2113, 989, 942, 3308, 2322, 2113, 3343, 3959, 3489]]

As can be seen that sentences are of different lengths. Hence padding will be done to make them of equal lenghts.

## Embedding Representation

In [0]:
embedded_docs = pad_sequences(onehot_rep, padding='pre', maxlen=20)

In [23]:
embedded_docs[:5]

array([[   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,  242,
        4462, 4210, 2930, 4913, 2641, 3230,  134, 1770, 3650],
       [   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0, 4007, 3917,  490, 2881, 2113, 2127, 3479],
       [   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0, 4905, 1399, 4612, 4964],
       [   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0, 1559, 2312, 1795, 4178, 2038, 2410],
       [   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,  171,
        2113,  989,  942, 3308, 2322, 2113, 3343, 3959, 3489]],
      dtype=int32)

In [24]:
len(embedded_docs)

18285

## Creating the Model

In [25]:
embedding_vector_features = 40
model = Sequential()
model.add(Embedding(vocab_size, embedding_vector_features, input_length = 20))
model.add(LSTM(100))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
print(model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 20, 40)            200000    
_________________________________________________________________
lstm (LSTM)                  (None, 100)               56400     
_________________________________________________________________
dense (Dense)                (None, 1)                 101       
Total params: 256,501
Trainable params: 256,501
Non-trainable params: 0
_________________________________________________________________
None


### Storing text and the labels as numpy arrays to feed the model

In [0]:
X_final = np.array(embedded_docs)
y_final = np.array(y)

In [0]:
from sklearn.model_selection import train_test_split

In [0]:
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, random_state=3, test_size=0.33)

## Model Training

In [31]:
model.fit(X_train, y_train, validation_data=(X_test, y_test),epochs=10, batch_size=64)

Epoch 1/10
192/192 [==============================] - 2s 9ms/step - loss: 0.3372 - accuracy: 0.8403 - val_loss: 0.1898 - val_accuracy: 0.9221
Epoch 2/10
192/192 [==============================] - 1s 7ms/step - loss: 0.1390 - accuracy: 0.9439 - val_loss: 0.1812 - val_accuracy: 0.9208
Epoch 3/10
192/192 [==============================] - 1s 7ms/step - loss: 0.1006 - accuracy: 0.9626 - val_loss: 0.2049 - val_accuracy: 0.9140
Epoch 4/10
192/192 [==============================] - 1s 7ms/step - loss: 0.0741 - accuracy: 0.9737 - val_loss: 0.2264 - val_accuracy: 0.9225
Epoch 5/10
192/192 [==============================] - 1s 7ms/step - loss: 0.0498 - accuracy: 0.9830 - val_loss: 0.2300 - val_accuracy: 0.9196
Epoch 6/10
192/192 [==============================] - 1s 7ms/step - loss: 0.0331 - accuracy: 0.9895 - val_loss: 0.2683 - val_accuracy: 0.9195
Epoch 7/10
192/192 [==============================] - 1s 7ms/step - loss: 0.0202 - accuracy: 0.9944 - val_loss: 0.4097 - val_accuracy: 0.9152
Epoch 

## Metrics

In [0]:
from sklearn.metrics import accuracy_score

In [34]:
y_pred = model.predict_classes(X_test)

Instructions for updating:
Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).


In [35]:
accuracy_score(y_test, y_pred)

0.9179784589892295

## Adding Dropout

In [0]:
from tensorflow.keras.layers import Dropout

In [43]:
embedding_vector_features = 40
model2 = Sequential()
model2.add(Embedding(vocab_size, embedding_vector_features, input_length = 20))
model2.add(LSTM(100))
model2.add(Dropout(0.))
model2.add(Dense(1,activation='sigmoid'))
model2.compile(loss='binary_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
print(model2.summary())

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_4 (Embedding)      (None, 20, 40)            200000    
_________________________________________________________________
lstm_4 (LSTM)                (None, 100)               56400     
_________________________________________________________________
dropout_4 (Dropout)          (None, 100)               0         
_________________________________________________________________
dense_4 (Dense)              (None, 1)                 101       
Total params: 256,501
Trainable params: 256,501
Non-trainable params: 0
_________________________________________________________________
None


In [44]:
model2.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=10, batch_size=64)

Epoch 1/10
192/192 [==============================] - 2s 9ms/step - loss: 0.3334 - accuracy: 0.8442 - val_loss: 0.1813 - val_accuracy: 0.9246
Epoch 2/10
192/192 [==============================] - 1s 7ms/step - loss: 0.1412 - accuracy: 0.9436 - val_loss: 0.1868 - val_accuracy: 0.9210
Epoch 3/10
192/192 [==============================] - 1s 7ms/step - loss: 0.0983 - accuracy: 0.9616 - val_loss: 0.1942 - val_accuracy: 0.9200
Epoch 4/10
192/192 [==============================] - 1s 7ms/step - loss: 0.0701 - accuracy: 0.9755 - val_loss: 0.2385 - val_accuracy: 0.9112
Epoch 5/10
192/192 [==============================] - 1s 7ms/step - loss: 0.0448 - accuracy: 0.9849 - val_loss: 0.2962 - val_accuracy: 0.9165
Epoch 6/10
192/192 [==============================] - 1s 7ms/step - loss: 0.0291 - accuracy: 0.9909 - val_loss: 0.3077 - val_accuracy: 0.9133
Epoch 7/10
192/192 [==============================] - 1s 7ms/step - loss: 0.0170 - accuracy: 0.9956 - val_loss: 0.3486 - val_accuracy: 0.9163
Epoch 